In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pulp
import random
import time

## User Input


In [ ]:
# print("Do you want personalized planning by providing some information like Height weight, etc.?")
print("If YES, Press X = 1, Otherwise X = 0")

x = int(input("X = "))


## Calculating BMR and Nutrients Constraints

In [ ]:
#BMR calculation 
def calculate_bmr(gender, weight, height, age):
    if gender.lower() == 'male':
        return (10 * weight) + (6.25 * height) - (5 * age) + 5
    else:
        return (10 * weight) + (6.25 * height) - (5 * age) - 161

if(x==1):
    
    # user = {
    # 'name': 'Jamiya Alom',
    # 'age': 20,
    # 'gender': 'female',
    # 'height': 190,  # in cm
    # 'weight': 75,   # in kg
    # }

    user = {}

    # user input
    user['name'] = input("Enter your name: ")
    user['age'] = int(input("Enter your age: "))
    user['gender'] = input("Enter your gender (male/female): ").lower()
    user['height'] = float(input("Enter your height in cm: "))
    user['weight'] = float(input("Enter your weight in kg: "))

    bmr1= calculate_bmr(user['gender'], user['weight'], user['height'], user['age'])
    print(f"Calculated BMR: {bmr1:.2f} kcal")

    min_cal= bmr1 * 1.2
    max_cal = bmr1 * 1.9

    # Minimum Calory needed
    bmr= bmr1*1.2

    # Protein Calculation
    Pconst = (bmr*0.20)/4

    # Fat Calculation
    Fconst = (bmr * 0.25)/9

    #Sugar Calculation
    Sconst = (bmr * 0.12)/4

    Sodium_const = 2500

    print(f"Minimum Calory: {bmr:.2f}, min_cal: {min_cal}, max_cal: {max_cal}, min Protein: {Pconst}, max_fat: {Fconst}, max_sugar: {Sconst}, max_sodium:{Sodium_const}")
    
else:
    min_cal = 2200
    max_cal = 3500
    Pconst = 110
    Fconst = 61
    Sconst = 66
    Sodium_const = 2500
    print(f"min_cal: {min_cal}, max_cal: {max_cal}, min Protein: {Pconst}, max_fat: {Fconst}, max_sugar: {Sconst}, max_sodium:{Sodium_const}")

### Dataset Loading


In [ ]:
#Data Loading
data = pd.read_csv("../dataset/7_11_food_choices.csv")
#data

### Data Extraction

In [ ]:
#Extracting the data and convert it to the list
prices     = data['Price(baht)'].tolist()
calories  = data['Calories(kCal)'].tolist()
proteins  = data['Protein(g)'].tolist()
fats      = data['Fat(g)'].tolist()
sugars    = data['Sugar(g)'].tolist()
sodium    = data['Sodium(mg)'].tolist()
items     = data['ITEM'].tolist()

n = len(data)  # Total food items

min_item = 3
max_item = 9

# Genetic Algorithm

In [ ]:
POPULATION_SIZE = 100
GENERATIONS = 300
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.1
ELITE_COUNT = 5

## Chromosome Creation

In [ ]:
def create_chromosome():
    for _ in range(1000): 
        chromosome = []
        for _ in range(n):
            # chromosome = [random.randint(0, 1) for _ in range(n)]
            # if min_item <= sum(chromosome):
            #     return chromosome
            if random.random() < 0.1:
                chromosome.append(1)
            else:
                chromosome.append(0)
        if min_item <= sum(chromosome) <= max_item:
            #print(f"\nChromosome selection...{chromosome}")
            return chromosome

    # If failed to create a valid one, force-select min items items
    chromosome = [0] * n
    selected_indices = random.sample(range(n), min_item)
    for idx in selected_indices:
        chromosome[idx] = 1
        #print("\nForce Chromosome selection...")
    return chromosome

## Fitness Function

In [ ]:
def fitness(chromosome):
    total_price = 0
    total_calories = 0
    total_protein = 0
    total_fat = 0
    total_sugar = 0
    total_sodium = 0
    item_count = 0

    for i in range(n):
        if chromosome[i]:
            total_price += prices[i]
            total_calories += calories[i]
            total_protein += proteins[i]
            total_fat += fats[i]
            total_sugar += sugars[i]
            total_sodium += sodium[i]
            item_count += 1

    penalty = 0
    if item_count < min_item or item_count > max_item:
        penalty += 100
    if total_calories < min_cal or total_calories > max_cal:
        penalty += 100
    if total_protein < Pconst:
        penalty += 100
    if total_fat > Fconst:
        penalty += 100
    if total_sugar > Sconst:
        penalty += 100
    if total_sodium > Sodium_const:
        penalty += 100

    return total_price + penalty

## Selection

In [ ]:
def select(population):
    candidates = random.sample(population, 3)
    return min(candidates, key=fitness)

## Crossover

In [ ]:
def crossover(parent1, parent2):
    """
    Single-point crossover
    """
    if random.random() < CROSSOVER_RATE:
        point = random.randint(1, n - 2)
        child1 = fix_chromosome(parent1[:point] + parent2[point:])
        child2 = fix_chromosome(parent2[:point] + parent1[point:])
        return [child1, child2]
    return [parent1[:], parent2[:]]

## Mutation

In [ ]:
def mutate(chromosome):
    """
    Randomly flips genes based on mutation rate, then fixes item count if needed.
    """
    for i in range(n):
        if random.random() < MUTATION_RATE:
            chromosome[i] = 1 - chromosome[i]
    return fix_chromosome(chromosome)

def fix_chromosome(chromosome):
    """
    Ensures that the number of selected items is within valid range.
    """
    while sum(chromosome) < min_item:
        index = random.randint(0, n - 1)
        chromosome[index] = 1
    while sum(chromosome) > max_item:
        ones = [i for i in range(n) if chromosome[i] == 1]
        chromosome[random.choice(ones)] = 0
    return chromosome

## Main Function

In [ ]:
NUM_RUNS = 13  # Number of GA runs
all_best_costs = []
all_avg_costs = []
all_execution_times = []

for run in range(NUM_RUNS):

    print(f"\n=== GA Run {run + 1} ===")

    start_time = time.time()

    print("\nRunning Genetic Algorithm...")

    # population = []
    # for _ in range(POPULATION_SIZE):
    #     population.append(create_chromosome())
    population = [create_chromosome() for _ in range(POPULATION_SIZE)]

    best_costs = []
    average_costs = []
    selects_cost = []

    for generation in range(GENERATIONS):
       
        new_population = sorted(population, key=fitness)[:ELITE_COUNT]  #Elitism

        best_fitness = fitness(new_population[0])
        best_costs.append(best_fitness)

        average_costs.append(sum(fitness(c) for c in population) / len(population))
    

        # Generate the rest of the population
        while len(new_population) < POPULATION_SIZE:
            parent1 = select(population)
            parent2 = select(population)
            offspring = crossover(parent1, parent2)
            for child in offspring:
                new_population.append(mutate(child))
    
        population = new_population


    print("\nBest Daily Meal Plan:")
    best_chromosome = min(population, key=fitness)

    total_cost = 0
    total_cal = 0
    total_protein = 0
    total_fat = 0
    total_sugar = 0
    total_sodium = 0

    for i in range(n):
        if best_chromosome[i]:
            print(f" - {items[i]} | {calories[i]} Kcal | Price: {prices[i]} Baht| Protein: {proteins[i]} g | Fat: {fats[i]} g | Sugar: {sugars[i]} g")
            total_cost += prices[i]
            total_cal += calories[i]
            total_protein += proteins[i]
            total_fat += fats[i]
            total_sugar += sugars[i]
            total_sodium += sodium[i]


    print("\nSummary of Best Meal Plan:")

   
    print(f"Total Cost: {total_cost:.2f} Baht")
    print(total_cal)

    if(total_cal >= min_cal and total_cal <= max_cal):
        print(f"\nTotal Cal: {total_cal:.2f} Kcal. ---- Calory Requirements: Satisfied :)")
    else: print(f"\nTotal Cal: {total_cal:.2f} Kcal. ---- Calory Requirements: not Satisfied :(")
    
    #Total Protein
    if(total_protein >= Pconst):
        print(f"\nTotal Protein: {total_protein:.2f} g. ---- Protein Requirements: Satisfied :)")
    else: print(f"\nTotal Protein: {total_protein:.2f} g. ---- Protein Requirements: Not Satisfied :(")

    #Total Fat
    if(total_fat <= Fconst):
        print(f"\nTotal fat: {total_fat:.2f} g. --------- Fat Requirements: Satisfied :)")
    else: print(f"\nTotal fat: {total_fat:.2f} g. --------- Fat Requirements: Not Satisfied :(")

    #Total Sugar
    if(total_sugar <= Sconst):
        print(f"\nTotal sugar: {total_sugar:.2f} g. ------- Sugar Requirements: Satisfied :)")
    else: print(f"\nTotal sugar: {total_sugar:.2f} g. ------- Sugar Requirements: Not Satisfied :(")    

    #Total Sodium
    if(total_sodium <= Sodium_const):
        print(f"\nTotal sodium: {total_sodium:.2f} mg. --- Sodium Requirements: Satisfied :)")
    else: print(f"\nTotal sodium: {total_sodium:.2f} mg. --- Sodium Requirements: Not Satisfied :(") 

    all_best_costs.append(min(best_costs))
    all_avg_costs.append(average_costs[-1])
    all_execution_times.append(time.time() - start_time)
    print(f"Best Cost: {min(best_costs):.2f}, Avg Cost: {average_costs[-1]:.2f}, Time: {all_execution_times[-1]:.2f}s")

    print("\n")

### GA all Run Summary

In [ ]:
print("\nGA all Run Summary-\n")
print(f"Min of All Best Cost over {NUM_RUNS} runs: {min(all_best_costs):.2f}")
print(f"Max of All Best Cost over {NUM_RUNS} runs: {max(all_best_costs):.2f}")
print(f"Average Best Cost over {NUM_RUNS} runs: {sum(all_best_costs)/NUM_RUNS:.2f}")
print(f"Average after {NUM_RUNS} runs: {sum(all_avg_costs)/NUM_RUNS:.2f}")
print(f"Average Execution Time: {sum(all_execution_times)/NUM_RUNS:.2f} seconds")

## Plotting

In [ ]:
#Plotting 
plt.plot(range(1, NUM_RUNS+1), all_best_costs, marker='o', label='Best Cost')
plt.plot(range(1, NUM_RUNS+1), all_avg_costs, marker='s', label='Final Avg Cost')
plt.title("GA Run Comparison")
plt.xlabel("Run Number")
plt.ylabel("Cost")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#Plotting Over Generations
plt.plot(best_costs, label='Best Cost per Generation', color='green')
plt.plot(average_costs, label='Average Cost per Generation', color='red')
plt.xlabel('Generation')
plt.ylabel('Cost (Fitness Value)')
plt.title('GA Optimization Progress')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()